In [1]:
import pandas as pd
import numpy as np

# 数据规整：连接、联合、重塑

## 1. 分层索引

#### Series分层索引

In [2]:
ser = pd.Series(np.random.randn(9), index = [['good','good','good','good','good','bad','bad','bad','bad'],
                                             ['a','a','a','b','b','b','b','c','c'],[1,2,3,1,2,3,1,2,3]])
ser

good  a  1   -2.000649
         2    1.012795
         3    0.201422
      b  1    1.206962
         2   -0.604302
bad   b  3    0.859061
         1    0.976125
      c  2   -0.410336
         3   -0.174849
dtype: float64

In [3]:
ser['good']['a']

1   -2.000649
2    1.012795
3    0.201422
dtype: float64

In [4]:
ser.loc[:,'c']

bad  2   -0.410336
     3   -0.174849
dtype: float64

In [10]:
ser.unstack()

1         2         3
bad  b  1.624100       NaN -0.340082
     c       NaN  0.369799 -0.044377
good a  0.389121  0.864123 -1.005931
     b  0.136535  0.061069       NaN

In [5]:
ser.index

MultiIndex(levels=[['bad', 'good'], ['a', 'b', 'c'], [1, 2, 3]],
           labels=[[1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 1, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

#### DataFrame分层索引

In [5]:
df = pd.DataFrame(np.arange(12).reshape((4,3)), index = [['a','a','b','b'],[1,2,1,2]], columns = [['APP','APP','Goole'],['Red','Bule','Red']])
df

APP      Goole
    Red Bule   Red
a 1   0    1     2
  2   3    4     5
b 1   6    7     8
  2   9   10    11

In [6]:
df.index.names = ['k1','k2']
df.columns.names = ['Company','Color']
df

Company APP      Goole
Color   Red Bule   Red
k1 k2                 
a  1      0    1     2
   2      3    4     5
b  1      6    7     8
   2      9   10    11

In [21]:
df.loc['a','APP']

Color,Red,Bule
k2,,
1,0,1
2,3,4


In [22]:
df.index

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['k1', 'k2'])

In [23]:
df.columns

MultiIndex(levels=[['APP', 'Goole'], ['Bule', 'Red']],
           labels=[[0, 0, 1], [1, 0, 1]],
           names=['Company', 'Color'])

## 2. 重排序和层级排序

sort_index只能在单一层级上对数据进行排序,通过level来控制排序的层级

In [7]:
df

Company APP      Goole
Color   Red Bule   Red
k1 k2                 
a  1      0    1     2
   2      3    4     5
b  1      6    7     8
   2      9   10    11

In [8]:
df.sort_index(level = 1)

Company APP      Goole
Color   Red Bule   Red
k1 k2                 
a  1      0    1     2
b  1      6    7     8
a  2      3    4     5
b  2      9   10    11

In [10]:
df.swaplevel(0,1).sort_index(level = 0)

Company APP      Goole
Color   Red Bule   Red
k2 k1                 
1  a      0    1     2
   b      6    7     8
2  a      3    4     5
   b      9   10    11

按层级进行汇总统计

In [33]:
df.sum(level = 0)

Company APP      Goole
Color   Red Bule   Red
k1                    
a         3    5     7
b        15   17    19

## 3.联合与合并数据集

merge()   

![微信图片_20190404204436.png](https://i.loli.net/2019/04/04/5ca5fc558e5b8.png)

### 多对一连接

In [12]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [13]:
df2 = pd.DataFrame({'key':['a','b','c'],'data2':range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,c


In [14]:
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,2


没有指定在那一列上连接，将自动重叠列名（相同的列名）作为重叠的键，可以使用参数 on 来显式的指定

In [15]:
pd.merge(df1, df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,2


如果每个对象的列名时不同的，需要分别为他们指定列名,而且在默认情况下，merge做的是内连接，结果的键是两个表的交集，通过参数 how = 'outer'改为两个表的并集

how的参数：
    
    - ‘inner’ 交集
    - ‘outer’ 并集
    - ‘left’  所有左表的键
    - ‘right’ 所有右表的键

In [16]:
df1 = pd.DataFrame({'key1':['b','b','a','c','a','a','b'],'data1':range(7)})
df1

,data1,key1
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [17]:
df2 = pd.DataFrame({'key2':['a','b','c'],'data2':range(3)})
df2

,data2,key2
0,0,a
1,1,b
2,2,c


In [18]:
pd.merge(df1, df2, left_on = 'key1', right_on = 'key2')

,data1,key1,data2,key2
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,3,c,2,c


### 多对多连接

In [27]:
df1 = pd.DataFrame({'key':['b','b','c','a','a','b'],'data1':range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,c
3,3,a
4,4,a
5,5,b


In [28]:
df2 = pd.DataFrame({'key':['a','b','a','b','c'],'data2':range(5)})
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,c


In [31]:
pd.merge(df1,df2,how = 'outer')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,c,4
7,3,a,0
8,3,a,2
9,4,a,0
